# Secure Data Access To S3 Bucket via IAM Policy using VPC

In [1]:
import boto3
import sagemaker

session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

s3 = boto3.Session().client(service_name='s3', region_name=region)
iam = boto3.Session().client(service_name='iam', region_name=region)

In [2]:
import time
timestamp = int(time.time())

In [3]:
bucket_secure_name = 'bucket-secure-iam-policy-vpc-{}'.format(timestamp)
print(bucket_secure_name)

bucket-secure-iam-policy-vpc-1608442211


In [4]:
!aws s3 mb s3://$bucket_secure_name

make_bucket: bucket-secure-iam-policy-vpc-1608442211


In [5]:
!aws s3 ls s3://$bucket_secure_name

# TODO: split to get name

In [6]:
#role_name = sagemaker.get_execution_role()
#print(role_name)
role_name='TeamRole'

In [7]:
policy_name='DSOAWS_SecureBucket_Policy_VPC_{}'.format(timestamp)
print(policy_name)

DSOAWS_SecureBucket_Policy_VPC_1608442211


In [8]:
# TODO:  Lock down to VPC

In [9]:
different_vpc_id='blah'

In [10]:
# Create the IAM policy
iam_policy_deny = {
        'Version': '2012-10-17',
        'Statement': [{
            'Sid': '',
            'Effect': 'Deny',
            'Action': [
                's3:ListBucket'
            ],
            'Resource': [
                'arn:aws:s3:::{}'.format(bucket_secure_name)
            ],
            'Condition': {
                'StringNotEquals': {
                    'aws:sourceVpce': different_vpc_id
                }
            }
            
        }]
    }

In [11]:
import json 

response = iam.put_role_policy(
    RoleName=role_name,
    PolicyName=policy_name,
    PolicyDocument=json.dumps(iam_policy_deny)
)

print(response)

time.sleep(5)

{'ResponseMetadata': {'RequestId': '1eeedba9-a1b5-4015-bf9f-8a56f6f5c003', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '1eeedba9-a1b5-4015-bf9f-8a56f6f5c003', 'content-type': 'text/xml', 'content-length': '206', 'date': 'Sun, 20 Dec 2020 05:30:30 GMT'}, 'RetryAttempts': 0}}


# TODO:  Check why this is still working.  LIkely because TeamRole has AdministratorAccess?

In [12]:
!aws s3 ls s3://$bucket_secure_name

In [13]:
response = iam.delete_role_policy(
    RoleName=role_name,
    PolicyName=policy_name
)
print(response)

{'ResponseMetadata': {'RequestId': '2ebf8217-3715-4e86-8e6e-5e385c634b3b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '2ebf8217-3715-4e86-8e6e-5e385c634b3b', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Sun, 20 Dec 2020 05:30:50 GMT'}, 'RetryAttempts': 0}}


In [14]:
!aws s3 ls s3://$bucket_secure_name